In [1]:
import pandas as pd
import regex as re
import msoffcrypto
import openpyxl
from io import BytesIO

In [3]:
#Load all the data into dataframes from our documents
censo = pd.read_csv(r"Comercial_Censo.csv",sep=";", encoding='latin1')
meteo=pd.read_csv(r"Comercial_Meteorología.csv")
com_test=pd.read_csv(r"Comercial_Test.csv",sep=";", encoding='latin1')
ventas=pd.read_csv(r"Comercial_Ventas.csv",sep=";", encoding='latin1')

#The excel document is encrypted, we need to decrypt it
file_path = r"Comercial_ParqueVehiculos1.xlsx"
password = "IndesIA2022!"

with open(file_path, 'rb') as file:
    encrypted_file = msoffcrypto.OfficeFile(file)
    encrypted_file.load_key(password=password)
    
    decrypted_file = BytesIO()
    encrypted_file.decrypt(decrypted_file)
    decrypted_file.seek(0)

    #We load the data from the excel file differenciating the years
    parque2016 = pd.read_excel(decrypted_file, engine='openpyxl', sheet_name=0, header=2)
    parque2017 = pd.read_excel(decrypted_file, engine='openpyxl', sheet_name=1, header=2)
    parque2018 = pd.read_excel(decrypted_file, engine='openpyxl', sheet_name=2, header=2)


In [4]:
#We apply some transformations to the data
meteo=meteo.drop(columns=["Unnamed: 0"])
meteo=meteo.replace("A CORUA","Coruña, A")
meteo["fecha"]=pd.to_datetime(meteo["fecha"])
meteo.set_index("fecha",inplace=True)
meteo.drop(columns=["provincia","indicativo","nombre"],inplace=True)

#com_test=com_test.drop(columns=["Unnamed: 0"])
#com_test["Dia"]=pd.to_datetime(com_test["Dia"])
#com_test.set_index("Dia",inplace=True)

parque2016.columns=parque2016.columns.str.replace("\n"," ")
parque2017.columns=parque2017.columns.str.replace("\n"," ")
parque2018.columns=parque2018.columns.str.replace("\n"," ")

def clean_name(text):
    return re.sub(r'^\d+\s*', '', text).strip()
censo['Municipio'] = censo['Municipio'].apply(clean_name)

ventas.set_index("Dia",inplace=True)
ventas["Fiesta"]= ventas["Fiesta"].apply(lambda x: 1 if pd.notna(x) else 0)
#ventas=pd.get_dummies(ventas,columns=["Weekday"],drop_first=True)

In [5]:
#We will create a subset dataframe with the data from the stations
stations=ventas.drop(columns=['Producto','PVP', 'VentasN','Day','Month', 'Year', 'Weekday', 'Fiesta'])
stations.drop_duplicates(inplace=True)
stations.reset_index(inplace=True)
stations=stations.drop(columns=["Dia"])
stations=stations.dropna()

# We check if there is another duplicates in the data by checking the coordinates
stations["Longitud"].duplicated()

stations.drop_duplicates(subset=["Latitud","Longitud"],inplace=True)

stations=stations.set_index("Nombre").sort_index()

stations

#(I think we should take out the coordinates out of the datasets, the value of the information is "included" on the Station itself)

,Latitud,Longitud
Nombre,,
ES1,"43,11401346","-8,14795451"
ES10,"43,37307585","-8,42789755"
ES11,"43,34128816","-8,20368109"
ES12,"43,47224306","-8,16188278"
ES2,"43,50287379","-8,22186654"
ES3,"43,27540082","-8,52940773"
ES4,"43,31748953","-8,47923782"
ES5,"43,31197376","-8,36521447"
ES6,"43,34600444","-8,26885678"


In [6]:
#Lets check the missing values in the ventas dataset
ventas.loc[ventas.isna().any(axis=1)]

#We take a look on the column PVP to analyze the possible cause od the missing values. Seeing that is a small amount of data we can drop it
pvpnan=ventas.loc[ventas["PVP"].isna()==True]

ventas=ventas.dropna()

In [7]:
#Lets check the missing values in the meteo dataset
meteo.isna().sum()

meteo.loc[meteo.isna().any(axis=1)]

meteo=meteo.dropna()


In [8]:
#Lets clean thhe parques dataset
parque2016=parque2016.drop(columns=["PROVINCIAS"])
parque2017=parque2017.drop(columns=["PROVINCIAS"])
parque2018=parque2018.drop(columns=["PROVINCIAS"])

#We rename the columns to avoid confusion when merging the dataframes 
parque2016=parque2016.add_suffix("_2016")
parque2017=parque2017.add_suffix("_2017")
parque2018=parque2018.add_suffix("_2018")

In [9]:
#This dataset has the Camiones and furgonetas merged, while the others it has them separated. We will merge them in the other datasets due we dont have the data about the individual types of vehicles
parque2016.columns

parque2017["CAMIONES y FURGONETAS_2017"]=parque2017["CAMIONES_2017"]+parque2017["FURGONETAS_2017"]
parque2017=parque2017.drop(columns=["CAMIONES_2017","FURGONETAS_2017"])

parque2018["CAMIONES y FURGONETAS_2018"]=parque2018["CAMIONES_2018"]+parque2018["FURGONETAS_2018"]
parque2018=parque2018.drop(columns=["CAMIONES_2018","FURGONETAS_2018"])

#Rename the columns to merge the datasets
parque2016 = parque2016.rename(columns={"MUNICIPIOS_2016": "MUNICIPIOS"})
parque2017 = parque2017.rename(columns={"MUNICIPIOS_2017": "MUNICIPIOS"})
parque2018 = parque2018.rename(columns={"MUNICIPIOS_2018": "MUNICIPIOS"})

#Lets create a single dataset with the data from the 3 years
parque_m=pd.merge(parque2016,parque2017,how="inner",on="MUNICIPIOS")
parque=pd.merge(parque_m,parque2018,how="inner",on="MUNICIPIOS")

parque.columns=parque.columns.str.capitalize()


In [10]:
#This dataset is the one for the testing
com_test


,Unnamed: 0,Dia,Producto,Latitud,Longitud,PVP,VentasN,Nombre
0,1,2019-01-01,95,"43,31748953","-8,47923782","1,199","0,0939126888063492",ES4
1,2,2019-01-01,GOA,"43,31748953","-8,47923782","1,159","0,233796029661409",ES4
2,3,2019-01-02,95,"43,31748953","-8,47923782","1,199","0,111861605268607",ES4
3,4,2019-01-02,GOA,"43,31748953","-8,47923782","1,159","0,510476600506945",ES4
4,5,2019-01-03,95,"43,31748953","-8,47923782","1,199","0,133608521518117",ES4
...,...,...,...,...,...,...,...,...
739,740,2019-01-29,GOA,"43,33886779","-8,39372721","1,219","0,457499321941958",ES7
740,741,2019-01-30,95,"43,33886779","-8,39372721","1,259","0,233729997287768",ES7
741,742,2019-01-30,GOA,"43,33886779","-8,39372721","1,219","0,624450772986168",ES7
742,743,2019-01-31,95,"43,33886779","-8,39372721","1,259","0,246619880661785",ES7


In [11]:
meteo

,altitud,dir,horaPresMax,horaPresMin,horaracha,horatmax,horatmin,prec,presMax,presMin,racha,sol,tmax,tmed,tmin,velmedia
fecha,,,,,,,,,,,,,,,,
2016-01-01,58,18,00,11,10:50,13:10,23:59,"7,0","1012,5","998,3","19,4","1,8","15,3","12,4","9,4","5,8"
2016-01-02,58,22,11,24,01:20,14:30,06:20,"2,2","1014,3","1006,2","13,6","5,7","14,0","11,2","8,4","4,2"
2016-01-03,58,25,00,13,13:20,Varias,00:00,"12,0","1006,2","996,9","19,2","0,0","15,6","14,0","12,3","5,8"
2016-01-04,58,29,Varias,14,15:00,00:00,23:40,"11,2","999,9","994,5","17,8","0,0","13,0","11,0","9,1","6,7"
2016-01-05,58,99,23,06,Varias,13:50,11:00,"7,6","1006,2","998,6","21,4","1,9","11,2","9,5","7,8","10,8"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-01-27,58,30,19,00,02:10,12:40,03:00,"3,2","1016,9","1011,9","16,7","4,3","12,7","11,2","9,8","5,0"
2019-01-28,58,29,00,24,06:30,11:50,00:20,"1,4","1016,3","1012,7","14,2","1,0","13,1","11,8","10,5","3,6"
2019-01-29,58,26,01,09,10:00,10:30,21:30,"4,6","1012,9","1004,6","19,4","0,5","12,3","10,4","8,4","6,9"


In [12]:
#import prophet
#m=prophet.Prophet()

In [13]:
meteo.head()

,altitud,dir,horaPresMax,horaPresMin,horaracha,horatmax,horatmin,prec,presMax,presMin,racha,sol,tmax,tmed,tmin,velmedia
fecha,,,,,,,,,,,,,,,,
2016-01-01,58,18,00,11,10:50,13:10,23:59,"7,0","1012,5","998,3","19,4","1,8","15,3","12,4","9,4","5,8"
2016-01-02,58,22,11,24,01:20,14:30,06:20,"2,2","1014,3","1006,2","13,6","5,7","14,0","11,2","8,4","4,2"
2016-01-03,58,25,00,13,13:20,Varias,00:00,"12,0","1006,2","996,9","19,2","0,0","15,6","14,0","12,3","5,8"
2016-01-04,58,29,Varias,14,15:00,00:00,23:40,"11,2","999,9","994,5","17,8","0,0","13,0","11,0","9,1","6,7"
2016-01-05,58,99,23,06,Varias,13:50,11:00,"7,6","1006,2","998,6","21,4","1,9","11,2","9,5","7,8","10,8"


In [14]:
meteo.isna().sum()

altitud        0
dir            0
horaPresMax    0
horaPresMin    0
horaracha      0
horatmax       0
horatmin       0
prec           0
presMax        0
presMin        0
racha          0
sol            0
tmax           0
tmed           0
tmin           0
velmedia       0
dtype: int64